In [0]:
import tensorflow as tf
print(tf.__version__)

1.14.0


In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('Gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at Gdrive


In [0]:
import os
import shutil

%cd /content/Gdrive/My Drive

if not os.path.exists('Tensorflow'):
  os.makedirs('Tensorflow')

%cd /content/Gdrive/My Drive/Tensorflow

#tensorflow models section has an object detection library
!git clone --quiet https://github.com/tensorflow/models.git

#this reposity has what it takes to prepare tfrecord files, bing thanks to Tony608
!git clone --quiet https://github.com/Tony607/object_detection_demo
 
#install PyDrive so we can upload our dataset from Google Drive
!pip install PyDrive

# protobuf is needed for creating py files from models library above
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

# !pip install -q Cython contextlib2 pillow lxml matplotlib

# pycoco for coco scores
!pip install -q pycocotools

# creating py files from protos
%cd /content/Gdrive/My Drive/Tensorflow/models/research
!protoc object_detection/protos/*.proto --python_out=.

# setting path, if not set, python can not use object detection library (from models)
os.environ['PYTHONPATH'] += ':/content/Gdrive/My Drive/Tensorflow/models/research/:/content/Gdrive/My Drive/Tensorflow/models/research/slim/'

!python object_detection/builders/model_builder_test.py

/content/Gdrive/My Drive
/content/Gdrive/My Drive/Tensorflow
fatal: destination path 'models' already exists and is not an empty directory.
fatal: destination path 'object_detection_demo' already exists and is not an empty directory.
     |████████████████████████████████| 993kB 3.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive
Selecting previously unselected package python-bs4.
(Reading database ... 130963 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselec

In [0]:
import os

%cd /content/Gdrive/My Drive/Tensorflow
if not os.path.exists('workspace'):
  os.makedirs('workspace')


%cd /content/Gdrive/My Drive/Tensorflow/workspace
%rm -rf training_dino

#####replace training_OPM1 with your training dir name#####
if not os.path.exists('training_dino'):
  os.makedirs('training_dino')

training_dino = '/content/Gdrive/My\ Drive/Tensorflow/workspace/training_dino'
%cd {training_dino}


fileId = '1y5J34231gI3ap0GUKc_fpMEsqAdXsE16'

from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Create GoogleDrive instance with authenticated GoogleAuth instance.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create GoogleDriveFile instance 
fileName = fileId + '.zip'
downloaded = drive.CreateFile({'id': fileId})
downloaded.GetContentFile(fileName)
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)


/content/Gdrive/My Drive/Tensorflow
/content/Gdrive/My Drive/Tensorflow/workspace
/content/Gdrive/My Drive/Tensorflow/workspace/training_dino


W0713 12:48:06.190999 139745989588864 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

Extracted zip file 1y5J34231gI3ap0GUKc_fpMEsqAdXsE16.zip


In [0]:
%cd {training_dino}

# modified from https://www.dlology.com/blog/how-to-train-an-object-detection-model-easy-for-free/

# Convert train folder annotation xml files to a single csv file,
!python /content/Gdrive/My\ Drive/Tensorflow/object_detection_demo/xml_to_csv.py \
-i train/Annotations -o train/train_labels.csv -l train/Annotations

!python /content/Gdrive/My\ Drive/Tensorflow/object_detection_demo/xml_to_csv.py \
-i test/Annotations -o test/test_labels.csv

/content/Gdrive/My Drive/Tensorflow/workspace/training_dino
Successfully converted xml to csv.
Generate `train/Annotations/label_map.pbtxt`
Successfully converted xml to csv.


In [0]:
training_dino = '/content/Gdrive/My\ Drive/Tensorflow/workspace/training_dino'
%cd {training_dino}

import pandas as pd

pd = pd.read_csv('train/train_labels.csv')
classes = sorted(pd['class'].unique())

#create pb file of label map
#mapping of your classes with its corresponding identification number (id) in 
with open('label_map.pbtxt', 'w') as txtl:
  for i, class_name in enumerate(classes):
    txtl.write("item {\n  id: %s\n  name: '%s'\n}\n\n"%(i+1, class_name))

with open('label_map.pbtxt') as txtl:
  print(txtl.read())  
    

/content/Gdrive/My Drive/Tensorflow/workspace/training_dino
item {
  id: 1
  name: 'Bird'
}

item {
  id: 2
  name: 'Cactus'
}

item {
  id: 3
  name: 'Dino'
}




In [0]:
%cd {training_dino}
%rm -rf tfrecord

if not os.path.exists('tfrecord'):
  os.makedirs('tfrecord')

  
# Generate `train.record`
!python /content/Gdrive/My\ Drive/Tensorflow/object_detection_demo/generate_tfrecord.py \
--csv_input=train/train_labels.csv \
--output_path=tfrecord/train.record \
--img_path=train/Images \
--label_map label_map.pbtxt

# Generate `val.record`
!python /content/Gdrive/My\ Drive/Tensorflow/object_detection_demo/generate_tfrecord.py \
--csv_input=test/test_labels.csv \
--output_path=tfrecord/val.record \
--img_path=test/Images \
--label_map label_map.pbtxt


/content/Gdrive/My Drive/Tensorflow/workspace/training_dino
W0713 12:48:27.438616 139733450860416 deprecation_wrapper.py:119] From /content/Gdrive/My Drive/Tensorflow/object_detection_demo/generate_tfrecord.py:134: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0713 12:48:27.439424 139733450860416 deprecation_wrapper.py:119] From /content/Gdrive/My Drive/Tensorflow/object_detection_demo/generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0713 12:48:27.901043 139733450860416 deprecation_wrapper.py:119] From /content/Gdrive/My Drive/Tensorflow/models/research/object_detection/utils/label_map_util.py:132: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/Gdrive/My Drive/Tensorflow/workspace/training_dino/tfrecord/train.record
W0713 12:48:32.104179 139815561107328 deprecation_wrapper.py:119] From /content/Gdrive/My Driv

In [0]:
%cd {training_dino}

import os
import shutil
import glob
from urllib.request import urlretrieve
import tarfile



#list of pretrained models can be found at:
#https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md
MODEL = 'ssdlite_mobilenet_v2_coco_2018_05_09'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'ssdlite_model'

if not (os.path.exists(MODEL_FILE)):
  urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

#remove zip file
os.remove(MODEL_FILE)

if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/Gdrive/My Drive/Tensorflow/workspace/training_dino


In [0]:
# modified from https://www.dlology.com/blog/how-to-train-an-object-detection-model-easy-for-free/

%cd {training_dino}

import re
import os

pipeline_fn = '/content/Gdrive/My Drive/Tensorflow/models/research/object_detection/samples/configs/ssdlite_mobilenet_v2_coco.config'
checkpoint = os.path.join('.', 'ssdlite_model/model.ckpt')
train_record = os.path.join('.', 'tfrecord/train.record')
val_record = os.path.join('.', 'tfrecord/val.record')
label_map = os.path.join('.', 'label_map.pbtxt')

# generate examples by augmenting existing images in tfrecords
num_eval = len(os.listdir('test/Images'))

# specify classes
num_class = len(classes)

# more steps of training gives higher accuracy
num_steps = 20000

#this specific arguably help speed up training process as it fits in Colabe's Tesla K80 GPU memory for selected model.
#I do not reccomend anything beyond size of 24(default) as I have tried, found training process significantly slower.
batch_size = 12
with open(pipeline_fn) as f:
    s = f.read()
    print(s)
with open(pipeline_fn, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(val_record), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_class), s)
    
    s = re.sub('num_examples: [0-9]+',
               'num_examples: {}'.format(num_eval), s)
    
    f.write(s)
    print(s)
    

/content/Gdrive/My Drive/Tensorflow/workspace/training_dino
# SSDLite with Mobilenet v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 3
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
     

In [0]:
%cd {training_dino}
         

pipeline_fn = '/content/Gdrive/My\ Drive/Tensorflow/models/research/object_detection/samples/configs/ssdlite_mobilenet_v2_coco.config'
obj_detection = '/content/Gdrive/My\ Drive/Tensorflow/models/research/object_detection'

!python {obj_detection + '/model_main.py'} \
    --pipeline_config_path={pipeline_fn}\
    --model_dir=fine_tuned_model \
    --alsologtostderr\
 
%ls

/content/Gdrive/My Drive/Tensorflow/workspace/training_dino
W0714 02:06:16.082246 140077666449280 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0714 02:06:17.056924 140077666449280 deprecation_wrapper.py:119] From /content/Gdrive/My Drive/Tensorflow/models/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0714 02:06:17.119185 140077666449280 deprecation_wrapper.py:119] From /content/Gdrive/My Drive/Tensorflow/models/research/slim/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0714 02:06:23.149827 140077666449280 

In [0]:
training_dino = '/content/Gdrive/My\ Drive/Tensorflow/workspace/training_dino'
obj_detection = '/content/Gdrive/My\ Drive/Tensorflow/models/research/object_detection'

%cd {training_dino}
%rm -rf Frozen_Inf


import re
import os
import numpy as np

#get the last model's check point
ckpt_files = [ckpt.replace('.meta', '') for ckpt in os.listdir('fine_tuned_model') if 'model.ckpt-' and '.meta' in ckpt]
steps = [(int(re.findall('\d+', ckpt_file)[0])) for ckpt_file in ckpt_files]
last_step = np.argmax(steps)
last_model = ckpt_files[last_step]

#exporting graph for testing
!python {obj_detection + '/export_inference_graph.py'} \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fn} \
    --output_directory=Frozen_Inf \
    --trained_checkpoint_prefix=fine_tuned_model/$last_model 
    
print('Finished exporting')

/content/Gdrive/My Drive/Tensorflow/workspace/training_dino
W0714 02:49:27.368729 139945191004032 deprecation_wrapper.py:119] From /content/Gdrive/My Drive/Tensorflow/models/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0714 02:49:27.432149 139945191004032 deprecation_wrapper.py:119] From /content/Gdrive/My Drive/Tensorflow/models/research/slim/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0714 02:49:27.479244 139945191004032 deprecation_wrapper.py:119] From /content/Gdrive/My Drive/Tensorflow/models/research/object_detection/export_inference_graph.py:156: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0714 02:49:27.479815 139945191004032 deprecation_wrapper.py:119] From /content/Gdrive/My Drive/Tensorflow/models/research/object_detection/export_inference_graph.py:139: The name tf.gfile.GFile is deprecated. Pl

In [0]:
training_dino = '/content/Gdrive/My\ Drive/Tensorflow/workspace/training_dino'
%cd {training_dino}

from google.colab import files
files.download('label_map.pbtxt')
files.download('Frozen_Inf/frozen_inference_graph.pb')

/content/Gdrive/My Drive/Tensorflow/workspace/training_dino
